# Gridded Data EMDA
## Ensemble of Meterological Dataset for North America 
### using probabilistc methods to estimate the uncertainty in spatial fields
https://essd.copernicus.org/articles/13/3337/2021/

1.	**Where is the observed data from?** *From weather stations on the ground, using statistical methods to fill in (spatially) daily data as a grid.* https://uofc-my.sharepoint.com/personal/heba_abdelmoaty_ucalgary_ca/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fheba%5Fabdelmoaty%5Fucalgary%5Fca%2FDocuments%2FUnfinished%5Fwork%2FCapstone%2FData&ga=1
2.	**What are the X and Y?**  *They are the grids overlaying the catchment. The X and Y of our project is in  245.25 and 49.55 respectively (so only need to use the one row).*
3.	**What are the 4 scenarios?** *Each scenario represents a predicted socio-economic status and the radiative energy that the globe will reach by the end of the century (4 socio-economic scenarios paired with radiative force we are producing (KJ/m2)).  **SSP1-2.6** = assumes reduced emissions, **SSP2-4.5** = assumes the same trend of emissions as historical, **SSP3-7.0** = medium to high emissions scenario, **SSP5-8.5**=optimal for economic development but with high emissions.*
4.	**What is the Trange?** *Trange = | Tmax – Tmin |    (probably wont need this)*


![Catchment Grid Location](CatchmentGrid_location_basemap.jpg)


# Install Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import linregress
from itertools import combinations
import math
from matplotlib.dates import MonthLocator, DateFormatter
import requests
from io import StringIO
import os

# Hard Coded Variables

In [2]:
observed_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/observed/obs_trange_final.csv"

## S1 Trange

In [3]:
s1_1_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_ACCESS-CM2_ssp126_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_2_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_BCC-CSM2-MR_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_3_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CanESM5_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_4_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CanESM5_ssp126_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_5_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CESM2_ssp126_r4i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s1_6_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CESM2_ssp126_r10i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s1_7_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CMCC-ESM2_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_8_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CNRM-CM6-1_ssp126_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_9_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CNRM-CM6-1-HR_ssp126_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_10_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_CNRM-ESM2-1_ssp126_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_11_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_EC-Earth3_ssp126_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_12_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_EC-Earth3_ssp126_r4i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_13_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_EC-Earth3-Veg_ssp126_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_14_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_EC-Earth3-Veg_ssp126_r2i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_15_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_FGOALS-g3_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_16_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_FGOALS-g3_ssp126_r3i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_17_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_GFDL-ESM4_ssp126_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_18_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s1_19_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_HadGEM3-GC31-MM_ssp126_r1i1p1f3_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s1_20_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_INM-CM4-8_ssp126_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_21_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_INM-CM5-0_ssp126_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_22_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_IPSL-CM6A-LR_ssp126_r2i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_23_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_KACE-1-0-G_ssp126_r1i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s1_24_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_KACE-1-0-G_ssp126_r2i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s1_25_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_MIROC6_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_26_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_MIROC6_ssp126_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_27_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_28_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_MPI-ESM1-2-HR_ssp126_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_29_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_MPI-ESM1-2-LR_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_30_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_MPI-ESM1-2-LR_ssp126_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_31_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_NESM3_ssp126_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_32_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_NESM3_ssp126_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s1_33_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_UKESM1-0-LL_ssp126_r1i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s1_34_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s1/reg_tas_day_UKESM1-0-LL_ssp126_r2i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"



## S2 Trange

In [4]:
s2_1_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_2_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_ACCESS-CM2_ssp245_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_3_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_4_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_5_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CanESM5_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_6_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CanESM5_ssp245_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_7_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CESM2_ssp245_r4i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s2_8_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CESM2_ssp245_r10i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s2_9_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CMCC-ESM2_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_10_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CNRM-CM6-1_ssp245_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_11_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_CNRM-ESM2-1_ssp245_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_12_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_EC-Earth3_ssp245_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_13_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_EC-Earth3_ssp245_r4i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_14_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_15_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_EC-Earth3-Veg_ssp245_r2i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_16_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_FGOALS-g3_ssp245_r3i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_17_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_GFDL-CM4_ssp245_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_18_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_GFDL-CM4_ssp245_r1i1p1f1_gr2_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_19_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_GFDL-ESM4_ssp245_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_20_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s2_21_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_INM-CM4-8_ssp245_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_22_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_INM-CM5-0_ssp245_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_23_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_IPSL-CM6A-LR_ssp245_r2i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_24_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_KACE-1-0-G_ssp245_r1i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s2_25_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_KACE-1-0-G_ssp245_r2i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s2_26_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MIROC6_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_27_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MIROC6_ssp245_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_28_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MPI-ESM1-2-HR_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_29_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MPI-ESM1-2-HR_ssp245_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_30_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MPI-ESM1-2-LR_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_31_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MPI-ESM1-2-LR_ssp245_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_32_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_MRI-ESM2-0_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_33_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_NESM3_ssp245_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_34_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_NESM3_ssp245_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s2_35_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_UKESM1-0-LL_ssp245_r1i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s2_36_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s2/reg_tas_day_UKESM1-0-LL_ssp245_r2i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"


## S3 Trange

In [5]:
s3_1_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_ACCESS-CM2_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_2_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_ACCESS-CM2_ssp370_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_3_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_AWI-CM-1-1-MR_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_4_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_BCC-CSM2-MR_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_5_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CanESM5_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_6_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CanESM5_ssp370_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_7_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CESM2_ssp370_r4i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s3_8_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CESM2_ssp370_r10i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s3_9_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CMCC-ESM2_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_10_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CNRM-CM6-1_ssp370_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_11_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_CNRM-ESM2-1_ssp370_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_12_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_EC-Earth3_ssp370_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_13_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_EC-Earth3_ssp370_r4i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_14_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_EC-Earth3-Veg_ssp370_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_15_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_EC-Earth3-Veg_ssp370_r4i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_16_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_FGOALS-g3_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_17_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_FGOALS-g3_ssp370_r3i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_18_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_GFDL-ESM4_ssp370_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_19_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_INM-CM4-8_ssp370_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_20_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_INM-CM5-0_ssp370_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_21_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_IPSL-CM6A-LR_ssp370_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_22_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_KACE-1-0-G_ssp370_r1i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s3_23_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_KACE-1-0-G_ssp370_r2i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s3_24_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MIROC6_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_25_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MIROC6_ssp370_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_26_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MPI-ESM1-2-HR_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_27_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MPI-ESM1-2-HR_ssp370_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_28_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MPI-ESM1-2-LR_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_29_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MPI-ESM1-2-LR_ssp370_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_30_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_MRI-ESM2-0_ssp370_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s3_31_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_UKESM1-0-LL_ssp370_r1i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s3_32_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s3/reg_tas_day_UKESM1-0-LL_ssp370_r2i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"


## S4 Trange

In [6]:
s4_1_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_ACCESS-CM2_ssp585_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_2_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_3_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_4_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CanESM5_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_5_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CanESM5_ssp585_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_6_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CESM2_ssp585_r4i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s4_7_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CESM2_ssp585_r10i1p1f1_gn_20150101-21010101_cannc_SPQM_07.nc.csv"
s4_8_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_9_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CNRM-CM6-1_ssp585_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_10_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_11_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_CNRM-ESM2-1_ssp585_r1i1p1f2_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_12_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_EC-Earth3_ssp585_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_13_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_EC-Earth3_ssp585_r4i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_14_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_15_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_EC-Earth3-Veg_ssp585_r4i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_16_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_FGOALS-g3_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_17_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_FGOALS-g3_ssp585_r3i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_18_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_19_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_GFDL-CM4_ssp585_r1i1p1f1_gr2_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_20_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_GFDL-ESM4_ssp585_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_21_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s4_22_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_HadGEM3-GC31-MM_ssp585_r1i1p1f3_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s4_23_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_INM-CM4-8_ssp585_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_24_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_INM-CM5-0_ssp585_r1i1p1f1_gr1_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_25_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_IPSL-CM6A-LR_ssp585_r2i1p1f1_gr_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_26_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_KACE-1-0-G_ssp585_r1i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s4_27_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_KACE-1-0-G_ssp585_r2i1p1f1_gr_20150101-21001230_cannc_SPQM_07.nc.csv"
s4_28_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MIROC6_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_29_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MIROC6_ssp585_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_30_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_31_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MPI-ESM1-2-HR_ssp585_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_32_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MPI-ESM1-2-LR_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_33_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MPI-ESM1-2-LR_ssp585_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_34_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_MRI-ESM2-0_ssp585_r1i2p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_35_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_NESM3_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_36_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_NESM3_ssp585_r2i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_37_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_NorESM2-LM_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_38_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_NorESM2-MM_ssp585_r1i1p1f1_gn_20150101-21001231_cannc_SPQM_07.nc.csv"
s4_39_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_UKESM1-0-LL_ssp585_r1i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"
s4_40_trange = "C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/s4/reg_tas_day_UKESM1-0-LL_ssp585_r2i1p1f2_gn_20150101-21001230_cannc_SPQM_07.nc.csv"


# Utility Functions

In [7]:
def filter_dataframe(df, x_value=245.25, y_value=49.55):
    """
    Filter a DataFrame to keep only rows where 'X' is equal to x_value and 'Y' is equal to y_value.

    Parameters:
    - df: pandas DataFrame
    - x_value: Value to match in the 'X' column
    - y_value: Value to match in the 'Y' column

    Returns:
    - pandas DataFrame containing filtered rows
    """
    filtered_df = df[(df['x'] == x_value) & (df['y'] == y_value)]
    return filtered_df

In [8]:
def reshape_dataframe(df, value_column_name):
    """
    Reshape a DataFrame from wide to long format by melting the date columns.

    Parameters:
    - df: pandas DataFrame with 'x', 'y', and date columns
    - value_column_name: Desired name for the value column ('tmean' or 'precip')

    Returns:
    - Reshaped pandas DataFrame with columns 'Date' and 'Value'
    """
    # Melt the DataFrame, keeping 'x' and 'y' as identifiers
    melted_df = pd.melt(df, id_vars=['x', 'y'], var_name='Date', value_name=value_column_name)

    # Remove 'X' from the beginning of 'Date' values, if present
    melted_df['Date'] = melted_df['Date'].str.replace('X', '')

    # Convert the 'Date' column to datetime format
    melted_df['Date'] = pd.to_datetime(melted_df['Date'], errors='coerce')  # Use errors='coerce' to handle invalid date formats

    return melted_df[['Date', value_column_name]]


In [25]:
def delete_empty_dataframes(dataframes_list):
    """
    Delete empty DataFrames from the given list.

    Parameters:
    - dataframes_list: List of DataFrames

    Returns:
    - List of DataFrames with empty DataFrames removed
    """
    non_empty_dataframes = [df for df in dataframes_list if not df.empty]
    return non_empty_dataframes


# Data Engineering

## Observed

In [9]:
obs_trange_df = pd.read_csv(observed_trange)
obs_trange_df = filter_dataframe(obs_trange_df)
obs_trange_df = reshape_dataframe(obs_trange_df, value_column_name='trange')
obs_trange_df.head()

,Date,trange
0,1979-01-01,12.577
1,1979-01-02,9.769
2,1979-01-03,9.818
3,1979-01-04,12.729
4,1979-01-05,13.063


In [10]:
csv_filename = "observed_trange.csv"  # Naming based on position in the list
csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/Observed/trange", csv_filename)
obs_trange_df.to_csv(csv_filepath, index=False)

## S1

In [11]:
s1_1_trange_df = pd.read_csv(s1_1_trange)
s1_2_trange_df = pd.read_csv(s1_2_trange)
s1_3_trange_df = pd.read_csv(s1_3_trange)
s1_4_trange_df = pd.read_csv(s1_4_trange)
s1_5_trange_df = pd.read_csv(s1_5_trange)
s1_6_trange_df = pd.read_csv(s1_6_trange) 
s1_7_trange_df = pd.read_csv(s1_7_trange)
s1_8_trange_df = pd.read_csv(s1_8_trange) 
s1_9_trange_df = pd.read_csv(s1_9_trange)
s1_10_trange_df = pd.read_csv(s1_10_trange)
s1_11_trange_df = pd.read_csv(s1_11_trange)
s1_12_trange_df = pd.read_csv(s1_12_trange)
s1_13_trange_df = pd.read_csv(s1_13_trange)
s1_14_trange_df = pd.read_csv(s1_14_trange)
s1_15_trange_df = pd.read_csv(s1_15_trange)
s1_16_trange_df = pd.read_csv(s1_16_trange)
s1_17_trange_df = pd.read_csv(s1_17_trange)
s1_18_trange_df = pd.read_csv(s1_18_trange)
s1_19_trange_df = pd.read_csv(s1_19_trange)
s1_20_trange_df = pd.read_csv(s1_20_trange)
s1_21_trange_df = pd.read_csv(s1_21_trange)
s1_22_trange_df = pd.read_csv(s1_22_trange)
s1_23_trange_df = pd.read_csv(s1_23_trange)
s1_24_trange_df = pd.read_csv(s1_24_trange)
s1_25_trange_df = pd.read_csv(s1_25_trange)
s1_26_trange_df = pd.read_csv(s1_26_trange)
s1_27_trange_df = pd.read_csv(s1_27_trange)
s1_28_trange_df = pd.read_csv(s1_28_trange)
s1_29_trange_df = pd.read_csv(s1_29_trange)
s1_30_trange_df = pd.read_csv(s1_30_trange)
s1_31_trange_df = pd.read_csv(s1_31_trange)
s1_32_trange_df = pd.read_csv(s1_32_trange)
s1_33_trange_df = pd.read_csv(s1_33_trange)
s1_34_trange_df = pd.read_csv(s1_34_trange)

In [12]:
dataframe_1_trange = [
    s1_1_trange_df, s1_2_trange_df, s1_3_trange_df, s1_4_trange_df, s1_5_trange_df,
    s1_6_trange_df, s1_7_trange_df, s1_8_trange_df, s1_9_trange_df, s1_10_trange_df,
    s1_11_trange_df, s1_12_trange_df, s1_13_trange_df, s1_14_trange_df, s1_15_trange_df,
    s1_16_trange_df, s1_17_trange_df, s1_18_trange_df, s1_19_trange_df, s1_20_trange_df,
    s1_21_trange_df, s1_22_trange_df, s1_23_trange_df, s1_24_trange_df, s1_25_trange_df,
    s1_26_trange_df, s1_27_trange_df, s1_28_trange_df, s1_29_trange_df, s1_30_trange_df,
    s1_31_trange_df, s1_32_trange_df, s1_33_trange_df, s1_34_trange_df
]
# Specify the value column name
value_column_name = 'trange'  

# Iterate over the list of dataframes
for i in range(len(dataframe_1_trange)):
    # Apply the filter_dataframe function
    dataframe_1_trange[i] = filter_dataframe(dataframe_1_trange[i])

    # Apply the reshape_dataframe function
    dataframe_1_trange[i] = reshape_dataframe(dataframe_1_trange[i], value_column_name)


In [13]:
dataframe_1_trange[0].head()

,Date,trange
0,2015-01-01,6.157924
1,2015-01-02,6.855071
2,2015-01-03,4.295697
3,2015-01-04,3.179259
4,2015-01-05,9.778684


In [26]:
non_empty_trange_dfs_s1= delete_empty_dataframes(dataframe_1_trange)

In [27]:
# Iterate over the merged dataframes for saving
for i, trange_df in enumerate(non_empty_trange_dfs_s1):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s1_{i+1}_trange.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S1/trange", csv_filename)
    trange_df.to_csv(csv_filepath, index=False)


## S2

In [15]:
s2_1_trange_df = pd.read_csv(s2_1_trange)
s2_2_trange_df = pd.read_csv(s2_2_trange)
s2_3_trange_df = pd.read_csv(s2_3_trange)
s2_4_trange_df = pd.read_csv(s2_4_trange)
s2_5_trange_df = pd.read_csv(s2_5_trange)
s2_6_trange_df = pd.read_csv(s2_6_trange) 
s2_7_trange_df = pd.read_csv(s2_7_trange)
s2_8_trange_df = pd.read_csv(s2_8_trange) 
s2_9_trange_df = pd.read_csv(s2_9_trange)
s2_10_trange_df = pd.read_csv(s2_10_trange)
s2_11_trange_df = pd.read_csv(s2_11_trange)
s2_12_trange_df = pd.read_csv(s2_12_trange)
s2_13_trange_df = pd.read_csv(s2_13_trange)
s2_14_trange_df = pd.read_csv(s2_14_trange)
s2_15_trange_df = pd.read_csv(s2_15_trange)
s2_16_trange_df = pd.read_csv(s2_16_trange)
s2_17_trange_df = pd.read_csv(s2_17_trange)
s2_18_trange_df = pd.read_csv(s2_18_trange)
s2_19_trange_df = pd.read_csv(s2_19_trange)
s2_20_trange_df = pd.read_csv(s2_20_trange)
s2_21_trange_df = pd.read_csv(s2_21_trange)
s2_22_trange_df = pd.read_csv(s2_22_trange)
s2_23_trange_df = pd.read_csv(s2_23_trange)
s2_24_trange_df = pd.read_csv(s2_24_trange)
s2_25_trange_df = pd.read_csv(s2_25_trange)
s2_26_trange_df = pd.read_csv(s2_26_trange)
s2_27_trange_df = pd.read_csv(s2_27_trange)
s2_28_trange_df = pd.read_csv(s2_28_trange)
s2_29_trange_df = pd.read_csv(s2_29_trange)
s2_30_trange_df = pd.read_csv(s2_30_trange)
s2_31_trange_df = pd.read_csv(s2_31_trange)
s2_32_trange_df = pd.read_csv(s2_32_trange)
s2_33_trange_df = pd.read_csv(s2_33_trange)
s2_34_trange_df = pd.read_csv(s2_34_trange)
s2_35_trange_df = pd.read_csv(s2_35_trange)
s2_36_trange_df = pd.read_csv(s2_36_trange)


In [16]:
dataframe_2_trange = [
    s2_1_trange_df, s2_2_trange_df, s2_3_trange_df, s2_4_trange_df, s2_5_trange_df,
    s2_6_trange_df, s2_7_trange_df, s2_8_trange_df, s2_9_trange_df, s2_10_trange_df,
    s2_11_trange_df, s2_12_trange_df, s2_13_trange_df, s2_14_trange_df, s2_15_trange_df,
    s2_16_trange_df, s2_17_trange_df, s2_18_trange_df, s2_19_trange_df, s2_20_trange_df,
    s2_21_trange_df, s2_22_trange_df, s2_23_trange_df, s2_24_trange_df, s2_25_trange_df,
    s2_26_trange_df, s2_27_trange_df, s2_28_trange_df, s2_29_trange_df, s2_30_trange_df,
    s2_31_trange_df, s2_32_trange_df, s2_33_trange_df, s2_34_trange_df, s2_35_trange_df, 
    s2_36_trange_df
]
# Specify the value column name
value_column_name = 'trange'  

# Iterate over the list of dataframes
for i in range(len(dataframe_2_trange)):
    # Apply the filter_dataframe function
    dataframe_2_trange[i] = filter_dataframe(dataframe_2_trange[i])

    # Apply the reshape_dataframe function
    dataframe_2_trange[i] = reshape_dataframe(dataframe_2_trange[i], value_column_name)


In [28]:
non_empty_trange_dfs_s2= delete_empty_dataframes(dataframe_2_trange)

In [29]:
# Iterate over the merged dataframes for saving
for i, trange_df in enumerate(non_empty_trange_dfs_s2):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s2_{i+1}_trange.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S2/trange", csv_filename)
    trange_df.to_csv(csv_filepath, index=False)


## S3

In [18]:
s3_1_trange_df = pd.read_csv(s3_1_trange)
s3_2_trange_df = pd.read_csv(s3_2_trange)
s3_3_trange_df = pd.read_csv(s3_3_trange)
s3_4_trange_df = pd.read_csv(s3_4_trange)
s3_5_trange_df = pd.read_csv(s3_5_trange)
s3_6_trange_df = pd.read_csv(s3_6_trange) 
s3_7_trange_df = pd.read_csv(s3_7_trange)
s3_8_trange_df = pd.read_csv(s3_8_trange) 
s3_9_trange_df = pd.read_csv(s3_9_trange)
s3_10_trange_df = pd.read_csv(s3_10_trange)
s3_11_trange_df = pd.read_csv(s3_11_trange)
s3_12_trange_df = pd.read_csv(s3_12_trange)
s3_13_trange_df = pd.read_csv(s3_13_trange)
s3_14_trange_df = pd.read_csv(s3_14_trange)
s3_15_trange_df = pd.read_csv(s3_15_trange)
s3_16_trange_df = pd.read_csv(s3_16_trange)
s3_17_trange_df = pd.read_csv(s3_17_trange)
s3_18_trange_df = pd.read_csv(s3_18_trange)
s3_19_trange_df = pd.read_csv(s3_19_trange)
s3_20_trange_df = pd.read_csv(s3_20_trange)
s3_21_trange_df = pd.read_csv(s3_21_trange)
s3_22_trange_df = pd.read_csv(s3_22_trange)
s3_23_trange_df = pd.read_csv(s3_23_trange)
s3_24_trange_df = pd.read_csv(s3_24_trange)
s3_25_trange_df = pd.read_csv(s3_25_trange)
s3_26_trange_df = pd.read_csv(s3_26_trange)
s3_27_trange_df = pd.read_csv(s3_27_trange)
s3_28_trange_df = pd.read_csv(s3_28_trange)
s3_29_trange_df = pd.read_csv(s3_29_trange)
s3_30_trange_df = pd.read_csv(s3_30_trange)
s3_31_trange_df = pd.read_csv(s3_31_trange)
s3_32_trange_df = pd.read_csv(s3_32_trange)


In [19]:
dataframe_3_trange = [
    s3_1_trange_df, s3_2_trange_df, s3_3_trange_df, s3_4_trange_df, s3_5_trange_df,
    s3_6_trange_df, s3_7_trange_df, s3_8_trange_df, s3_9_trange_df, s3_10_trange_df,
    s3_11_trange_df, s3_12_trange_df, s3_13_trange_df, s3_14_trange_df, s3_15_trange_df,
    s3_16_trange_df, s3_17_trange_df, s3_18_trange_df, s3_19_trange_df, s3_20_trange_df,
    s3_21_trange_df, s3_22_trange_df, s3_23_trange_df, s3_24_trange_df, s3_25_trange_df,
    s3_26_trange_df, s3_27_trange_df, s3_28_trange_df, s3_29_trange_df, s3_30_trange_df,
    s3_31_trange_df, s3_32_trange_df
]
# Specify the value column name
value_column_name = 'trange'  

# Iterate over the list of dataframes
for i in range(len(dataframe_3_trange)):
    # Apply the filter_dataframe function
    dataframe_3_trange[i] = filter_dataframe(dataframe_3_trange[i])

    # Apply the reshape_dataframe function
    dataframe_3_trange[i] = reshape_dataframe(dataframe_3_trange[i], value_column_name)


In [30]:
non_empty_trange_dfs_s3= delete_empty_dataframes(dataframe_3_trange)

In [33]:
# Iterate over the merged dataframes for saving
for i, trange_df in enumerate(non_empty_trange_dfs_s3):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s3_{i+1}_trange.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S3/trange", csv_filename)
    trange_df.to_csv(csv_filepath, index=False)


## S4

In [21]:
s4_1_trange_df = pd.read_csv(s4_1_trange)
s4_2_trange_df = pd.read_csv(s4_2_trange)
s4_3_trange_df = pd.read_csv(s4_3_trange)
s4_4_trange_df = pd.read_csv(s4_4_trange)
s4_5_trange_df = pd.read_csv(s4_5_trange)
s4_6_trange_df = pd.read_csv(s4_6_trange) 
s4_7_trange_df = pd.read_csv(s4_7_trange)
s4_8_trange_df = pd.read_csv(s4_8_trange) 
s4_9_trange_df = pd.read_csv(s4_9_trange)
s4_10_trange_df = pd.read_csv(s4_10_trange)
s4_11_trange_df = pd.read_csv(s4_11_trange)
s4_12_trange_df = pd.read_csv(s4_12_trange)
s4_13_trange_df = pd.read_csv(s4_13_trange)
s4_14_trange_df = pd.read_csv(s4_14_trange)
s4_15_trange_df = pd.read_csv(s4_15_trange)
s4_16_trange_df = pd.read_csv(s4_16_trange)
s4_17_trange_df = pd.read_csv(s4_17_trange)
s4_18_trange_df = pd.read_csv(s4_18_trange)
s4_19_trange_df = pd.read_csv(s4_19_trange)
s4_20_trange_df = pd.read_csv(s4_20_trange)
s4_21_trange_df = pd.read_csv(s4_21_trange)
s4_22_trange_df = pd.read_csv(s4_22_trange)
s4_23_trange_df = pd.read_csv(s4_23_trange)
s4_24_trange_df = pd.read_csv(s4_24_trange)
s4_25_trange_df = pd.read_csv(s4_25_trange)
s4_26_trange_df = pd.read_csv(s4_26_trange)
s4_27_trange_df = pd.read_csv(s4_27_trange)
s4_28_trange_df = pd.read_csv(s4_28_trange)
s4_29_trange_df = pd.read_csv(s4_29_trange)
s4_30_trange_df = pd.read_csv(s4_30_trange)
s4_31_trange_df = pd.read_csv(s4_31_trange)
s4_32_trange_df = pd.read_csv(s4_32_trange)
s4_33_trange_df = pd.read_csv(s4_33_trange)
s4_34_trange_df = pd.read_csv(s4_34_trange)
s4_35_trange_df = pd.read_csv(s4_35_trange)
s4_36_trange_df = pd.read_csv(s4_36_trange)
s4_37_trange_df = pd.read_csv(s4_37_trange)
s4_38_trange_df = pd.read_csv(s4_38_trange)
s4_39_trange_df = pd.read_csv(s4_39_trange)
s4_40_trange_df = pd.read_csv(s4_40_trange)


In [22]:
dataframe_4_trange = [
    s4_1_trange_df, s4_2_trange_df, s4_3_trange_df, s4_4_trange_df, s4_5_trange_df,
    s4_6_trange_df, s4_7_trange_df, s4_8_trange_df, s4_9_trange_df, s4_10_trange_df,
    s4_11_trange_df, s4_12_trange_df, s4_13_trange_df, s4_14_trange_df, s4_15_trange_df,
    s4_16_trange_df, s4_17_trange_df, s4_18_trange_df, s4_19_trange_df, s4_20_trange_df,
    s4_21_trange_df, s4_22_trange_df, s4_23_trange_df, s4_24_trange_df, s4_25_trange_df,
    s4_26_trange_df, s4_27_trange_df, s4_28_trange_df, s4_29_trange_df, s4_30_trange_df,
    s4_31_trange_df, s4_32_trange_df, s4_33_trange_df, s4_34_trange_df, s4_35_trange_df,
    s4_36_trange_df, s4_37_trange_df, s4_38_trange_df, s4_39_trange_df, s4_40_trange_df
]
# Specify the value column name
value_column_name = 'trange'  

# Iterate over the list of dataframes
for i in range(len(dataframe_4_trange)):
    # Apply the filter_dataframe function
    dataframe_4_trange[i] = filter_dataframe(dataframe_4_trange[i])

    # Apply the reshape_dataframe function
    dataframe_4_trange[i] = reshape_dataframe(dataframe_4_trange[i], value_column_name)


In [35]:
non_empty_trange_dfs_s4= delete_empty_dataframes(dataframe_4_trange)

In [36]:
# Iterate over the merged dataframes for saving
for i, trange_df in enumerate(non_empty_trange_dfs_s4):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s4_{i+1}_trange.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S4/trange", csv_filename)
    trange_df.to_csv(csv_filepath, index=False)
